# Ejercicio 4 - Análisis de texto

# Materia: Análisis y Curación de Datos -- Autor: Marcelo Cena

Para este ejercicio bajaremos el libro Cuentos de amor, locura y muerte de Horacio Quiroga, descargado de 'http://www.gutenberg.org/cache/epub/13507/pg13507.txt'.

Descripcion de lo requerido.
* Utilizar un texto de [proyecto Gutenberg en castellano](http://www.gutenberg.org/browse/languages/es) 
* Contar palabras y ordenar por frecuencia:
    * Limpiar preludio y licencia de Project Gutenberg
    * Omitir “palabras vacías” (stop words) y símbolos
* Encontrar personajes
* Hacer un análisis extra a gusto
* Hint: Tutorial para español: [PLN Python](https://relopezbriega.github.io/blog/2017/09/23/procesamiento-del-lenguaje-natural-con-python/)
    * Usa [spacy.io](https://spacy.io/) y [github.com/chartbeat-labs/textacy](https://github.com/chartbeat-labs/textacy)
    * No hace falta la parte de Deep Learning

En nuestro caso, analizamos el libro limpio y sin limpiar , pretendimos realizar un diagrama de nube d epalabras que no funcionó x problemas de compatibilidad (estamos tratando de resolver ese problema) y luego buscamos distintos tipos de palabras: sustantivos, adjetivos, verbos, etc, así como por su característica

In [111]:
# Importando las librerías que vamos a utilizar
import os
from os import path
import numpy as np 
import matplotlib.pyplot as plt 
import spacy
import textacy
import textacy.datasets as txdt
from collections import Counter
import string
from string import punctuation
#from wordcloud import WordCloud
#import wordcloud as wc


In [112]:
# Cargando el modelo en español de spacy
import es_core_news_sm
nlp = es_core_news_sm.load()
#nlp = spacy.load('es')
from spacy.lang.es.stop_words import STOP_WORDS

#stop_words = open('../input/spanish.txt', encoding="utf8", errors='ignore').read()
stop_words = stop_words.lower()
punctuation = punctuation + '¿¡'
punctuation = punctuation + '\r'

print("# Stopwords = {}".format(len(STOP_WORDS)))
print(STOP_WORDS)

# Stopwords = 551
{'fue', 'existe', 'general', 'siguiente', 'cuanta', 'seis', 'ultimo', 'soyos', 'mías', 'buenas', 'sabeis', 'mío', 'debajo', 'empleo', 'puede', 'ir', 'él', 'porque', 'llegó', 'empleais', 'respecto', 'hacer', 'primer', 'ustedes', 'sino', 'lado', 'tres', 'última', 'cuando', 'mas', 'enseguida', 'una', 'voy', 'dejó', 'fueron', 'muchos', 'mismas', 'dio', 'estados', 'cierto', 'intentar', 'alrededor', 'éste', 'comentó', 'quiere', 'casi', 'vamos', 'podrian', 'aproximadamente', 'después', 'dicen', 'estan', 'somos', 'algún', 'hasta', 'haciendo', 'delante', 'ésos', 'propio', 'cuanto', 'ése', 'hacerlo', 'nunca', 'solos', 'podria', 'aquélla', 'tercera', 'ser', 'dice', 'conocer', 'propios', 'otros', 'mí', 'bastante', 'donde', 'uso', 'explicó', 'nuevas', 'otro', 'nuestro', 'poner', 'nadie', 'ésa', 'bien', 'informó', 'mucha', 'tras', 'usamos', 'añadió', 'bajo', 'podemos', 'pueda', 'intenta', 'todavía', 'más', 'apenas', 'quiénes', 'señaló', 'pesar', 'dan', 'despacio', 'pasado', 'atras'

In [113]:
start = 0
stop = 0
import codecs
# Ubicamos el fin del preludio y el comienzo de la licencia de proyecto Gutemberg 
with codecs.open('../input/Cuentos_Quiroga.txt', "r", encoding="utf-8", errors='ignore') as file:
    for num, line in enumerate(file, 1):
        if 'START OF THIS PROJECT GUTENBERG EBOOK' in line:
            print('Licencia de Proyecto Gutemberg finaliza en línea:', num)
            start = num
        if '*** END OF THIS PROJECT GUTENBERG EBOOK CUENTOS DE AMOR DE LOCURA Y ***' in line:
            print('Libro finaliza en línea: ', num)
            stop = num
            
        
with codecs.open('../input/Cuentos_Quiroga.txt', "r", encoding="utf-8", errors='ignore') as file:
    # Leemos el archivo, lo dividimos en saltos de línea y seleccionamos solo las lineas que nos interesan
    texto_limpio = file.read().split('\n')[start:stop]
    # Convertimos el texto de list a string, reemplazamos saltos de linea por espacios
    texto_limpio2 = '\n'.join(texto_limpio).replace('\n',' ')
    texto = texto_limpio2.replace('\r','')
    texto_limpio3=nlp(texto)

Licencia de Proyecto Gutemberg finaliza en línea: 19
Libro finaliza en línea:  7103


In [114]:
texto_limpio3

    Produced by Stan Goodman, Paz Barrios and the Online Distributed Proofreading Team.       #Cuentos de Amor de Locura y de Muerte#   HORACIO QUIROGA  1917     #INDICE#   Una estación de amor Los ojos sombríos El solitario La muerte de Isolda El infierno artificial La gallina degollada Los buques suicidantes El almohadón de pluma El perro rabioso A la deriva La insolación El alambre de púa Los Mensú Yaguaí Los pescadores de vigas La miel silvestre Nuestro primer cigarro La meningitis y su sombra          #UNA ESTACION DE AMOR#     #Primavera#   Era el martes de carnaval. Nébel acababa de entrar en el corso, ya al oscurecer, y mientras deshacía un paquete de serpentinas, miró al carruaje de delante. Extrañado de una cara que no había visto la tarde anterior, preguntó a sus compañeros:   --¿Quién es? No parece fea.  --¡Un demonio! Es lindísima. Creo que sobrina, o cosa así, del doctor Arrizabalaga. Llegó ayer, me parece...  Nébel fijó entonces atentamente los ojos en la hermosa criatur

In [115]:
texto_limpio2

'\r \r \r \r Produced by Stan Goodman, Paz Barrios and the Online Distributed\r Proofreading Team.\r \r \r \r \r \r \r #Cuentos de Amor de Locura y de Muerte#\r \r \r HORACIO QUIROGA\r \r 1917\r \r \r \r \r #INDICE#\r \r \r Una estación de amor\r Los ojos sombríos\r El solitario\r La muerte de Isolda\r El infierno artificial\r La gallina degollada\r Los buques suicidantes\r El almohadón de pluma\r El perro rabioso\r A la deriva\r La insolación\r El alambre de púa\r Los Mensú\r Yaguaí\r Los pescadores de vigas\r La miel silvestre\r Nuestro primer cigarro\r La meningitis y su sombra\r \r \r \r \r \r \r \r \r \r #UNA ESTACION DE AMOR#\r \r \r \r \r #Primavera#\r \r \r Era el martes de carnaval. Nébel acababa de entrar en el corso, ya al\r oscurecer, y mientras deshacía un paquete de serpentinas, miró al\r carruaje de delante. Extrañado de una cara que no había visto la tarde\r anterior, preguntó a sus compañeros:\r \r \r --¿Quién es? No parece fea.\r \r --¡Un demonio! Es lindísima. Creo q

In [116]:
texto

'    Produced by Stan Goodman, Paz Barrios and the Online Distributed Proofreading Team.       #Cuentos de Amor de Locura y de Muerte#   HORACIO QUIROGA  1917     #INDICE#   Una estación de amor Los ojos sombríos El solitario La muerte de Isolda El infierno artificial La gallina degollada Los buques suicidantes El almohadón de pluma El perro rabioso A la deriva La insolación El alambre de púa Los Mensú Yaguaí Los pescadores de vigas La miel silvestre Nuestro primer cigarro La meningitis y su sombra          #UNA ESTACION DE AMOR#     #Primavera#   Era el martes de carnaval. Nébel acababa de entrar en el corso, ya al oscurecer, y mientras deshacía un paquete de serpentinas, miró al carruaje de delante. Extrañado de una cara que no había visto la tarde anterior, preguntó a sus compañeros:   --¿Quién es? No parece fea.  --¡Un demonio! Es lindísima. Creo que sobrina, o cosa así, del doctor Arrizabalaga. Llegó ayer, me parece...  Nébel fijó entonces atentamente los ojos en la hermosa criatu

In [117]:
mitexto_limpio= texto.lower()

mitexto_limpio = ''.join(c for c in mitexto_limpio if c not in punctuation)
mitexto_limpio = ' '.join([word for word in mitexto_limpio.split() if word not in stop_words])
mitexto_limpio

'produced by goodman paz barrios the online distributed proofreading team cuentos amor locura muerte horacio quiroga 1917 indice estación amor ojos sombríos solitario muerte isolda infierno artificial gallina degollada buques suicidantes almohadón pluma perro rabioso deriva insolación alambre púa mensú yaguaí pescadores vigas miel silvestre primer cigarro meningitis sombra estacion amor primavera martes carnaval nébel acababa entrar corso oscurecer mientras deshacía paquete serpentinas miró carruaje delante extrañado cara había visto tarde anterior preguntó compañeros quién parece fea demonio lindísima creo sobrina cosa así doctor arrizabalaga llegó ayer parece nébel fijó entonces atentamente ojos hermosa criatura chica joven aún acaso más catorce años completamente núbil tenía bajo cabello oscuro rostro suprema blancura blanco mate raso patrimonio exclusivo cutis finos ojos azules largos perdiéndose hacia sienes cerco negras pestañas acaso separados bajo frente tersa aire mucha noblez

In [118]:
# Procesamos texto con el modelo de Español de Spacy 
texto_procesado = nlp(mitexto_limpio)

In [119]:
# Seleccionar palabras que no son stop words o signos de puntuacion
words = [token.text for token in texto_procesado if token.is_stop != True and token.is_punct != True]
stop = [token.text for token in texto_procesado if token.is_stop == True or token.is_punct == True]

In [120]:
# Contamos la cantidad de palabras más utilizadas
Counter(words).most_common()

[('ojos', 147),
 ('nébel', 107),
 ('noche', 102),
 ('maría', 94),
 ('hombre', 74),
 ('fué', 74),
 ('cabeza', 72),
 ('casa', 72),
 ('mujer', 65),
 ('amor', 63),
 ('madre', 62),
 ('mamá', 56),
 ('años', 52),
 ('lidia', 52),
 ('perros', 51),
 ('monte', 50),
 ('mañana', 47),
 ('voz', 47),
 ('rato', 47),
 ('hora', 46),
 ('vida', 45),
 ('mano', 45),
 ('miró', 42),
 ('mirada', 41),
 ('seguida', 40),
 ('perro', 40),
 ('diez', 38),
 ('agua', 38),
 ('iba', 37),
 ('caballos', 37),
 ('volvió', 36),
 ('instante', 36),
 ('elvira', 35),
 ('pasó', 35),
 ('ah', 33),
 ('camino', 33),
 ('vió', 33),
 ('sol', 32),
 ('podía', 32),
 ('míster', 32),
 ('kassim', 32),
 ('fiebre', 31),
 ('paso', 30),
 ('allá', 30),
 ('cosa', 30),
 ('luz', 30),
 ('toro', 29),
 ('cama', 29),
 ('metros', 29),
 ('dicha', 27),
 ('hijo', 27),
 ('pie', 27),
 ('visto', 27),
 ('boca', 27),
 ('alma', 27),
 ('cayé', 26),
 ('mensú', 26),
 ('yaguaí', 26),
 ('manos', 26),
 ('joven', 25),
 ('cuerpo', 25),
 ('podeley', 25),
 ('río', 25),
 ('hac

In [131]:
#from wordcloud import WordCloud

#wordcloudx  = WordCloud(width=800,height=400,max_font_size=80,background_color="white",max_words=500,colormap="ocean").generate(filtered_text)
#plt.figure()
#plt.imshow(wordcloudx, interpolation="gaussian")
#plt.axis("off")
#plt.show()

#wordcloud no funcionó.... x la version de windows ... haremos otros cálculos...
from textacy.text_stats import TextStats
from textacy.doc import Doc
from textacy.corpus import Corpus

stats = TextStats(texto_procesado)
stats.basic_counts

{'n_chars': 170133,
 'n_long_words': 11394,
 'n_monosyllable_words': 3581,
 'n_polysyllable_words': 11354,
 'n_sents': 3,
 'n_syllables': 66085,
 'n_unique_words': 8563,
 'n_words': 26449}

In [152]:
# Trabajando con las entidades del texto
# Una entidad nombrada es cualquier objeto del mundo real como una persona,
# ubicación, organización o producto con un nombre propio.

# tipos de entidades del texto
set(ent.label_ for ent in texto_procesado.ents)

set()

el sistema no logra identificar entidades nombradas xq está todo en minúscula y sin conectores.....

In [164]:
# Obtenemos los personajes 
def personajes(txt):
    perso = Counter()
    for ent in txt.ents:
        if ent.label_ =='PERSON' or ent.label_=='PER' :
            perso[ent.lemma_] +=1
    return perso.most_common()

########

a = []



print(personajes (texto_procesado))

[]


Aquí pasa lo mismo, no encuentra nada porque no puede identificar entidades nombradas.

vamos con otro texto sin limpiar ..


In [166]:
mitexto_limpio3 = ''.join(c for c in texto if c not in punctuation)
mitexto_limpio3 = ' '.join([word for word in mitexto_limpio3.split() if word not in stop_words])
mitexto_limpio3

'Produced by Stan Goodman Paz Barrios the Online Distributed Proofreading Team Cuentos Amor Locura Muerte HORACIO QUIROGA 1917 INDICE Una estación amor Los ojos sombríos El solitario La muerte Isolda El infierno artificial La gallina degollada Los buques suicidantes El almohadón pluma El perro rabioso A deriva La insolación El alambre púa Los Mensú Yaguaí Los pescadores vigas La miel silvestre Nuestro primer cigarro La meningitis sombra UNA ESTACION DE AMOR Primavera Era martes carnaval Nébel acababa entrar corso oscurecer mientras deshacía paquete serpentinas miró carruaje delante Extrañado cara había visto tarde anterior preguntó compañeros Quién No parece fea Un demonio Es lindísima Creo sobrina cosa así doctor Arrizabalaga Llegó ayer parece Nébel fijó entonces atentamente ojos hermosa criatura Era chica joven aún acaso más catorce años completamente núbil Tenía bajo cabello oscuro rostro suprema blancura blanco mate raso patrimonio exclusivo cutis finos Ojos azules largos perdiéndo

In [168]:
stats = TextStats(texto_limpio3)
stats.basic_counts

{'n_chars': 229481,
 'n_long_words': 11655,
 'n_monosyllable_words': 23924,
 'n_polysyllable_words': 11647,
 'n_sents': 3081,
 'n_syllables': 92366,
 'n_unique_words': 8671,
 'n_words': 49839}

In [169]:
set(ent.label_ for ent in texto_limpio3.ents)

{'LOC', 'MISC', 'ORG', 'PER'}

In [163]:
# Obtenemos los personajes 
def personajes(txt):
    perso = Counter()
    for ent in txt.ents:
        if ent.label_ =='PERSON' or ent.label_=='PER' :
            perso[ent.lemma_] +=1
    return perso.most_common()

########

a = []



print(personajes (texto_limpio3)[:30])

[('', 157), ('Nébel', 55), ('Podeley', 23), ('Cayé', 17), ('María Elvira', 17), ('Luis María', 16), ('Candiyú', 16), ('Berta', 15), ('Mazzini', 15), ('Había', 13), ('Kassim', 13), ('Fué', 12), ('Cooper', 11), ('Benincasa', 10), ('Alicia', 10), ('Inés', 10), ('Alfonso', 9), ('Castelhum', 9), ('Después', 9), ('Jordán', 8), ('Así', 8), ('Allí', 7), ('Funes', 7), ('-PRON-', 7), ('Posadas', 6), ('¡ No', 6), ('Volví', 6), ('Prince', 6), ('Jones', 6), ('Milk', 5)]


In [161]:
[token.orth_ for token in texto_limpio3 if token.pos_ == 'NOUN']

['and',
 '1917',
 'estación',
 'amor',
 'ojos',
 'muerte',
 'gallina',
 'buques',
 'almohadón',
 'pluma',
 'deriva',
 'insolación',
 'pescadores',
 'vigas',
 'miel',
 'cigarro',
 'sombra',
 'martes',
 'carnaval',
 'corso',
 'paquete',
 'serpentinas',
 'carruaje',
 'cara',
 'tarde',
 'compañeros',
 'demonio',
 'sobrina',
 'cosa',
 'doctor',
 'ojos',
 'criatura',
 'chica',
 'años',
 'cabello',
 'rostro',
 'mate',
 'patrimonio',
 'cutis',
 'Ojos',
 'sienes',
 'cerco',
 'pestañas',
 'frente',
 'aire',
 'nobleza',
 'terquedad',
 'ojos',
 'semblante',
 'flor',
 'luz',
 'belleza',
 'sentirlos',
 'momento',
 'rodilla',
 'almohadón',
 'surrey',
 'momento',
 'serpentinas',
 'victoria',
 'carruajes',
 'puente',
 'cintas',
 'vez',
 'galante',
 'falta',
 'respeto',
 'personas',
 'carruaje',
 'hombro',
 'cabeza',
 'guardabarros',
 'serpentinas',
 'personas',
 'derrochador',
 'voz',
 'doctor',
 'conoces',
 'madre',
 'chica',
 'doctor',
 'pos',
 'examen',
 'señora',
 'exuberancia',
 'juventud',
 'debe

In [171]:
[token.orth_ for token in texto_limpio3 if token.pos_ == 'PROPN'][1:30]

['by',
 'Stan',
 'Goodman',
 'Paz',
 'Barrios',
 'the',
 'Online',
 'Distributed',
 'Proofreading',
 'Team',
 '#',
 'Cuentos',
 'Amor',
 'Locura',
 'Muerte',
 '#',
 'HORACIO',
 'QUIROGA',
 '#',
 'INDICE',
 '#',
 'Isolda',
 'infierno',
 'perro',
 'alambre',
 'Mensú',
 'Yaguaí',
 'meningitis',
 '#']

Como podemos ver, en este caso, al trabajar con el texto sin elimirar mayúsculas ni conectores, se pueden identificar algunos de los personajes, mezclados con otras palabras, como Fué, Allí, ¡No, Había, Después... evidentemente, es necesario pulir el diccionario españpl utilizado.